# Exploratory Data Analysis (EDA)

### Datathon challenge task
Predict the duration of time it takes for patients to receive metastatic cancer diagnosis.

### EDA Techniques
- Analysis
    - Load data
    - Understand the data:
        - Categorize the types of features
        - Data shape, types, missing (null) values, unique values
        - Categorical vs numeric data, and which of the categorical data is ordinal (has an order)
        - Statistical analysis
        - Visualizations
            - Univariate
            - Bivariate
            - Multivariate
    - Feature correlations
- Cleaning
    - Removing features
    - Handling missing data
    - Encoding categorical variables
- Feature engineering


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#pip install seaborn

In [ ]:
# Import libraries

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, RobustScaler
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style="whitegrid")

# Analysis

## Load data

In [ ]:
# Load .csv data as dataframes and make patient_id the index

df_train = pd.read_csv('data/train.csv', index_col='patient_id')
df_test = pd.read_csv('data/test.csv', index_col='patient_id')

# Look at the first few rows of data
df_train.head()

## Understand the data

In order to work with the data we need to know the different data types. Following are some basic types:

1. **Numerical**: Data represented by numbers and can be measured or expressed in number format.
   
       - Discrete Data: Data that can only take specific numerical values within a certain range, typically integers.
   
       - Continuous Data: Data that can take any value within a certain range and can be measured at any point on a scale.
   
       - Time Series Data: Data collected or recorded at regular time intervals, often used for analyzing trends over time.
   
   
3. **Categorical Data**: Data that represents categories or labels and cannot be measured on a numerical scale.
   
       - Ordinal Data: Data with categories that have a specific order or ranking.
   
       - Nominal Data: A type of categorical data where categories are merely labels without any inherent order.
   
   
5. **Others**: Spatial, unstructured free text, video, images, graph data
 

### Categorize the types of features

Read the [data descriptions](https://www.kaggle.com/competitions/widsdatathon2024-challenge2/data) on Kaggle.

**Patient characteristics:** patient_race, payer_type, patient_age, patient_gender, bmi

**Patient location:** patient_state, patient_zip3, region, division

**Breast cancer diagnosis information:** breast_cancer_diagnosis_code, breast_cancer_diagnosis_desc, metastatic_cancer_diagnosis_code, 
    metastatic_first_novel_treatment, metastatic_first_novel_treatment_type

**Geo (zip-code level) demographic data:** Many! (population, income, education, rent, race, poverty etc)

**Climate data:** 72 columns showing the zip 3 Monthly Average Temperature for the patient’s zip 3 and month referenced

**Target variable:** metastatic_diagnosis_period

In [ ]:
# Define a function that shows the data dimensions (# rows and columns), total # NA values, and data types, 
# number of distinct NA (null) values for each column (feature)

def initial_eda(df):
    if isinstance(df, pd.DataFrame):
        total_na = df.isna().sum().sum()
        print("Dimensions : %d rows, %d columns" % (df.shape[0], df.shape[1]))
        print("Total NA Values : %d " % (total_na))
        print("%38s %10s     %10s %10s" % ("Column Name", "Data Type", "#Distinct", "NaN Values"))
        col_name = df.columns
        dtyp = df.dtypes
        uniq = df.nunique()
        na_val = df.isna().sum()
        for i in range(len(df.columns)):
            print("%38s %10s   %10s %10s" % (col_name[i], dtyp[i], uniq[i], na_val[i]))
        
    else:
        print("Expect a DataFrame but got a %15s" % (type(df)))

In [ ]:
# Call the function on our training data

initial_eda(df_train)

In [ ]:
# Call the function on our testing data

initial_eda(df_test)

#### Observations

**Data types**

- 11 object types in train and test - these are categorical features
    - None of these seem to be ordinal

- 3 integer types in train, 2 in test - zip, age, and the target variable (metastatic_diagnosis_period)

- The rest of the features are floats

**# distinct values**

- Only 1 distinct value for patient_gender (female) in train and test data, so we can drop this feature

- Only 1 distinct value for metastatic_first_novel_treatment_type in train and test data, and only 11 rows contain a value in train and 7 in test, so we can drop this feature

- Only 2 distinct values for metastatic_first_novel_treatment in train and test data, and only 11 rows contain a value in train and 7 in test, so we can probably drop this feature, but look for any strong correlation with the target feature

- patient_zip3 and population have the same number of unique values in train (751) and test (669), so do we need them both? 
Population is a size, so it might be important. Is zip random? How strong is the correlation with the target variable?

**NaN values - need to start thinking about how to handle these**

- patient_race: ~half are missing in train and test

- payer_type: 13-14% are missing in train and test

- bmi: 69-70% are missing in train and test

- metastatic_cancer_diagnosis_code and metastatic_first_novel_treatment_type: almost all are missing

- Several features (23) are missing 5 values in the train data but none of those are missing in the test data. Are the 5 missing values for
each feature in the same row? If so, can we just drop those rows? (Yes, all are in the same rows and they are in the same zip, and those
are the only 5 records for that zip, but that zip is not in the test data, so we can probably drop them.)

- Avg temps: Missing 0-180 in train and 0-95 in test. The 180/95 are in Apr-14 and are all in NY (6 zips) or ID (1 zip) in both train
and test


### Drop features and observations that logically add no value

In [ ]:
# Drop features

# Only one value for patient_gender
# Only one value for metastatic_first_novel_treatment_type and two values for metastatic_first_novel_treatment, 
# and there are only 11 non-null rows in the train data, and 7 non-null in the test data

drop_features = [
                    'metastatic_first_novel_treatment',         # rarely filled in
                    'metastatic_first_novel_treatment_type',    # rarely filled in
                    'patient_gender',                           # always the same value
                   ]

# Always perform the same actions on train and test data
df_train = df_train.drop(drop_features, axis=1)
df_test = df_test.drop(drop_features, axis=1)


In [ ]:
# Drop rows

# Drop the 5 rows that have 23 missing features all from the same zip; that zip is not present in the test set
mask = (df_train['patient_zip3'] == 772)
df_train = df_train.drop(df_train[df_train['patient_zip3'] == 772].index)

# Check the shape (originally we had 13173 rows)
df_train.shape

### Statistical summary

In [ ]:
# Do not truncate

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
print(df_train.describe())

In [ ]:
print(df_test.describe())

#### Observations

**patient_zip3** max in test is different than train so we know there are values in test that are not in train (and probably vice versa)

**patient_age** range from 18-91

**bmi** max is 97 in train but only 43.7 in test! Looking closer, there is one 90 and one 97 in the train data, which are anomalies,
perhaps we remove those rows

**target variable** min=0, max=365, mean=96.5, std=109, count of 0=3126, count >= 350=224

#### Distribution plots of the variables are a good way to understand differences in data representation. Ideally we want the train and test data sets to have similar distributions as they both represent a random sample of the true population.


### Categorical vs numerical features

In [ ]:
# Create variables for the categorical and numeric features of df_train (df_test will be the same except the target col)
# Also create a variable for the target column

cat_cols=df_train.select_dtypes(include=['object']).columns
num_cols = df_train.select_dtypes(include=np.number).columns.tolist()
target_col = 'metastatic_diagnosis_period'
print("Categorical Variables:")
print(cat_cols)
print("Numerical Variables:")
print(num_cols)

### Compare unique values in df_train and df_test

Which features are present in the training data but not in the test data, and vice versa

In [ ]:
# Iterate over the columns of df_train
for col in df_train.columns:
    # Check if the column exists in df_test
    if col in df_test.columns:
        # Get the unique values of col in df_train
        unique_values_train = df_train[col].unique()
        # Get the unique values of col in df_test
        unique_values_test = df_test[col].unique()
        # Find the differences between the two sets
        differences = set(unique_values_train) - set(unique_values_test)
        diff_count = len(differences)
        if len(differences) > 0:
            print(f"{col} has the following unique values that are not present in the test dataset: {differences}")
            print(f"{col} has the following number of differences: {diff_count}")

In [ ]:
# Iterate over the columns of df_test
for col in df_test.columns:
    # Check if the column exists in df_test
    if col in df_train.columns:
        # Get the unique values of col in df_train
        unique_values_test = df_test[col].unique()
        # Get the unique values of col in df_test
        unique_values_train = df_train[col].unique()
        # Find the differences between the two sets
        differences = set(unique_values_test) - set(unique_values_train)
        diff_count = len(differences)
        if len(differences) > 0:
            print(f"{col} has the following unique values that are not present in the train dataset: {differences}")
            print(f"{col} has the following number of differences: {diff_count}")

#### Observations

**patient_race, payer_type, patient_state, Region, Division, patient_age** Same values in train and test

**patient_zip3 and population** Train has 93 zips/populations not in test, test has 12 zips/populations not in train

**breast_cancer_diagnosis_code** Train has 7 values not in test (but just 1 or a few occurences of each), test has 1 value not in train (but only 1 occurrence of it)

**metastatic_cancer_diagnosis_code** Train has 7 values not in test (but just 1 or a few occurences of each), test has 2 values not in train (but only 1 occurrence of each)

### Visualizations

Categorical variables can be visualized using a Count plot, Bar Chart, Pie Plot, etc.

Numerical Variables can be visualized using Histogram, Box Plot, Density Plot, etc.

Box Plot: Also marks the outliers. the first box edge or line represents 25%(first quartile) of the data is below it. The middle line is the median, the third the end of the box represents 75%(third quartiles) of the data values. Q3-Q1 is known as the Interquartile range (IQR) and values lower than 1.5 times the Q1 and values higher than 1.5 times the Q3 are typically considered outliers.

#### Univariate analysis

Univariate analysis examines the distribution of a single variable in isolation, without considering its relationship with other variables. It helps to understand the characteristics and patterns within individual variables, including measures of central tendency, dispersion, and shape of the distribution.


In [ ]:
# Histogram and box plot of numerical data for df_train

for col in num_cols:
    print(col)
    print('Skew :', round(df_train[col].skew(), 2))
    plt.figure(figsize = (15, 4))
    plt.subplot(1, 2, 1)
    df_train[col].hist(grid=False)
    plt.ylabel('count')
    plt.subplot(1, 2, 2)
    sns.boxplot(x=df_train[col])
    plt.show()

In [ ]:
# Histogram and box plot of numerical data for df_test

for col in num_cols:
    print(col)
    print('Skew :', round(df_test[col].skew(), 2))
    plt.figure(figsize = (15, 4))
    plt.subplot(1, 2, 1)
    df_test[col].hist(grid=False)
    plt.ylabel('count')
    plt.subplot(1, 2, 2)
    sns.boxplot(x=df_test[col])
    plt.show()

#### Observations of data

**bmi** 3 major outliers for train data, perhaps we drop those rows

**density** is quite skewed and has many outliers, but same for both data sets

**metastatic_diagnosis_period** Median is less than 50 days, has a right positive skew

**age_over_80** Out of proportion between the data sets based on histograms (box plots are very similar)


In [ ]:
print(cat_cols)

In [ ]:
# Count plot of categorical data for df_train

fig, axes = plt.subplots(4, 2, figsize = (18, 18))
fig.suptitle('Bar plot for all categorical variables in the dataset')
sns.countplot(ax = axes[0, 0], x = 'patient_race', data = df_train, color = 'blue', 
              order = df_train['patient_race'].value_counts().index);
sns.countplot(ax = axes[0, 1], x = 'payer_type', data = df_train, color = 'blue', 
              order = df_train['payer_type'].value_counts().index);
sns.countplot(ax = axes[1, 0], x = 'patient_state', data = df_train, color = 'blue', 
              order = df_train['patient_state'].value_counts().index);
sns.countplot(ax = axes[1, 1], x = 'Region', data = df_train, color = 'blue', 
              order = df_train['Region'].value_counts().index);
sns.countplot(ax = axes[2, 0], x = 'Division', data = df_train, color = 'blue', 
              order = df_train['Division'].value_counts().index);
sns.countplot(ax = axes[2, 1], x = 'breast_cancer_diagnosis_code', data = df_train, color = 'blue', 
              order = df_train['breast_cancer_diagnosis_code'].value_counts().index);
sns.countplot(ax = axes[3, 0], x = 'breast_cancer_diagnosis_desc', data = df_train, color = 'blue', 
              order = df_train['breast_cancer_diagnosis_desc'].value_counts().index);
sns.countplot(ax = axes[3, 1], x = 'metastatic_cancer_diagnosis_code', data = df_train, color = 'blue', 
              order = df_train['metastatic_cancer_diagnosis_code'].value_counts().index);

axes[1][0].tick_params(labelrotation=90);
axes[2][0].tick_params(labelrotation=45);
axes[2][1].tick_params(labelrotation=90);
axes[3][0].tick_params(labelrotation=90);
axes[3][1].tick_params(labelrotation=90);

In [ ]:
# Count plot of categorical data for df_test

fig, axes = plt.subplots(4, 2, figsize = (18, 18))
fig.suptitle('Bar plot for all categorical variables in the dataset')
sns.countplot(ax = axes[0, 0], x = 'patient_race', data = df_test, color = 'blue', 
              order = df_test['patient_race'].value_counts().index);
sns.countplot(ax = axes[0, 1], x = 'payer_type', data = df_test, color = 'blue', 
              order = df_test['payer_type'].value_counts().index);
sns.countplot(ax = axes[1, 0], x = 'patient_state', data = df_test, color = 'blue', 
              order = df_test['patient_state'].value_counts().index);
sns.countplot(ax = axes[1, 1], x = 'Region', data = df_test, color = 'blue', 
              order = df_test['Region'].value_counts().index);
sns.countplot(ax = axes[2, 0], x = 'Division', data = df_test, color = 'blue', 
              order = df_test['Division'].value_counts().index);
sns.countplot(ax = axes[2, 1], x = 'breast_cancer_diagnosis_code', data = df_test, color = 'blue', 
              order = df_test['breast_cancer_diagnosis_code'].value_counts().index);
sns.countplot(ax = axes[3, 0], x = 'breast_cancer_diagnosis_desc', data = df_test, color = 'blue', 
              order = df_test['breast_cancer_diagnosis_desc'].value_counts().index);
sns.countplot(ax = axes[3, 1], x = 'metastatic_cancer_diagnosis_code', data = df_test, color = 'blue', 
              order = df_test['metastatic_cancer_diagnosis_code'].value_counts().index);

axes[1][0].tick_params(labelrotation=90);
axes[2][0].tick_params(labelrotation=45);
axes[2][1].tick_params(labelrotation=90);
axes[3][0].tick_params(labelrotation=90);
axes[3][1].tick_params(labelrotation=90);


In [ ]:
# Group the data by breast_cancer_diagnosis_code and calculate the mean metastatic_diagnosis_period

grouped_data = df_train.groupby("breast_cancer_diagnosis_code").mean()["metastatic_diagnosis_period"]
grouped_data.head()

In [ ]:
# Plot a bar chart of the mean metastatic_diagnosis_period for each breast_cancer_diagnosis_code

plt.figure(figsize=(15, 8))
plt.bar(range(len(grouped_data)), grouped_data)
plt.xticks(range(len(grouped_data)), grouped_data.index, rotation=90)
plt.xlabel("Diagnosis Code")
plt.ylabel("Mean Metastatic Diagnosis Period")
plt.title("Mean Metastatic Diagnosis Period for Each Breast Cancer Diagnosis Code")
plt.show()

In [ ]:
# Group the data by metastatic_cancer_diagnosis_code and calculate the mean metastatic_diagnosis_period

grouped_data = df_train.groupby("metastatic_cancer_diagnosis_code").mean()["metastatic_diagnosis_period"]
grouped_data.head()

In [ ]:
# Plot a bar chart of the mean metastatic_diagnosis_period for each metastatic_cancer_diagnosis_code

plt.figure(figsize=(15, 8))
plt.bar(range(len(grouped_data)), grouped_data)
plt.xticks(range(len(grouped_data)), grouped_data.index, rotation=90)
plt.xlabel("Diagnosis Code")
plt.ylabel("Mean Metastatic Diagnosis Period")
plt.title("Mean Metastatic Diagnosis Period for Each Metastatic Cancer Diagnosis Code")
plt.show()

## Visualizations - bivariate analysis

Bivariate analysis explores the relationship between two variables to understand how they interact or influence each other. This highlights any dependencies or trends between two variables.

Cons: Often too simplistic and ignores potential confounding factors or nonlinear associations. Does not capture complex relations.

Note for other methods of analysis:

1. Multivariate Analysis: Examines the relationship between multiple independent variables and a single dependent variable.
2. Factor Analysis: Explores underlying factors that explain the correlations among observed variables.
3. Cluster analysis: Identifies groups or clusters of observations with similar characteristics based on multiple variables.

In [ ]:
print(df_train['breast_cancer_diagnosis_code'].unique())

In [ ]:
# Use bar plot to show the relationship between Categorical variables and the target continuous variables 

fig, axarr = plt.subplots(4, 2, figsize=(18, 24))
df_train.groupby('patient_race')[target_col].mean().sort_values(ascending=False).plot.bar(ax=axarr[0][0], fontsize=12)
axarr[0][0].set_title("patient_race Vs target", fontsize=18)
df_train.groupby('payer_type')[target_col].mean().sort_values(ascending=False).plot.bar(ax=axarr[0][1], fontsize=12)
axarr[0][1].set_title("payer_type Vs target", fontsize=18)
df_train.groupby('patient_state')[target_col].mean().sort_values(ascending=False).head(20).plot.bar(ax=axarr[1][0], fontsize=12)
axarr[1][0].set_title("patient_state top 10", fontsize=18)
df_train.groupby('Region')[target_col].mean().sort_values(ascending=False).plot.bar(ax=axarr[1][1], fontsize=12)
axarr[1][1].set_title("Region Vs target", fontsize=18)
df_train.groupby('Division')[target_col].mean().sort_values(ascending=False).plot.bar(ax=axarr[2][0], fontsize=12)
axarr[2][0].set_title("Division Vs target", fontsize=18)
df_train.groupby('breast_cancer_diagnosis_code')[target_col].mean().sort_values(ascending=False).plot.bar(ax=axarr[2][1], fontsize=12)
axarr[2][1].set_title("breast_cancer_diagnosis_code Vs target", fontsize=18)
df_train.groupby('breast_cancer_diagnosis_desc')[target_col].mean().sort_values(ascending=False).plot.bar(ax=axarr[3][0], fontsize=12)
axarr[3][0].set_title("breast_cancer_diagnosis_desc Vs target", fontsize=18)
df_train.groupby('metastatic_cancer_diagnosis_code')[target_col].mean().sort_values(ascending=False).plot.bar(ax=axarr[3][1], fontsize=12)
axarr[3][1].set_title("metastatic_cancer_diagnosis_code Vs target", fontsize=18)
plt.subplots_adjust(hspace=1.0)
plt.subplots_adjust(wspace=.5)
sns.despine()

In [ ]:


plt.figure(figsize=(15, 8))
df_train.groupby('patient_state')[target_col].mean().sort_values(ascending=False).plot.bar(fontsize=12)
plt.show()

## Clean data

In [ ]:
# Data prep - replace categorical column nulls with 'unknown'

for column in cat_cols:
    df_train[column] = df_train[column].mask(df_train[column].isnull(), 'unknown')
    df_test[column] = df_test[column].mask(df_test[column].isnull(), 'unknown')

In [ ]:
# Drop target from num_cols

num_cols.remove(target_col)

In [ ]:
# Data prep - remaining nulls are in numeric columns - convert them to mean values from df_train
# (since df_train has more data and will likely have a more representative mean value than df_test)

for column in num_cols:
    mean_value = df_train[column].mean()
    df_train[column] = df_train[column].mask(df_train[column].isnull(), mean_value)
    df_test[column] = df_test[column].mask(df_test[column].isnull(), mean_value) 

df_train.head()

# Alternatively, assign large numbers to nulls in numeric columns
'''
for column in num_cols:
    replacement_value = 1000000
    df_train[column] = df_train[column].mask(df_train[column].isnull(), replacement_value)
    df_test[column] = df_test[column].mask(df_test[column].isnull(), replacement_value) 

df_train.head()
'''

In [ ]:
# Convert categorical columns to numbers using LabelEncoder()

for column in cat_cols:
    df_combined = pd.concat([df_train, df_test], axis=0)
    le = LabelEncoder()
    le.fit(df_combined[column])
    df_train[column] = le.transform(df_train[column])
    df_test[column] = le.transform(df_test[column])

df_train.head()

In [ ]:
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

In [ ]:
df_test.head()

## Correlations

Correlation coefficients quantify the strength and direction of the linear relationship between two variables. Positive correlations indicate that as one variable increases, the other tends to increase as well, while negative correlations suggest that as one variable increases, the other tends to decrease.

### Correlation EDA by feature category

Since there are so many features, a heatmap of all features is not comprehensible. So let us break down the features in a way that makes sense for the analysis.

In [ ]:
patient_demographics = [
    "patient_race",
    "patient_state",
    "patient_zip3",
    "patient_age",
    "bmi",
    "metastatic_diagnosis_period"
]

diagnosis_codes = [
    "breast_cancer_diagnosis_code",
    "metastatic_cancer_diagnosis_code"
]

treatment = [
    "metastatic_first_novel_treatment",
    "metastatic_first_novel_treatment_type"
]

demographics_by_zip_code = [
    "population",
    "density",
    "age_median",
    "male",
    "female",
    "married",
    "family_size",
    "income_household_median",
    "income_household_six_figure",
    "home_ownership",
    "housing_units",
    "home_value",
    "rent_median",
    "education_college_or_above",
    "labor_force_participation",
    "unemployment_rate",
    "metastatic_diagnosis_period"
]

race_and_ethnicity = [
    "race_white",
    "race_black",
    "race_asian",
    "race_native",
    "race_pacific",
    "race_other",
    "race_multiple",
    "hispanic",
    "metastatic_diagnosis_period"
]

age_groups = [
    "age_under_10",
    "age_10_to_19",
    "age_20s",
    "age_30s",
    "age_40s",
    "age_50s",
    "age_60s",
    "age_70s",
    "age_over_80", 
    "metastatic_diagnosis_period"
]

marital_status = [
    "divorced",
    "never_married",
    "widowed",
    "metastatic_diagnosis_period"
]

income = [
    "family_dual_income",  # Create this feature based on family_dual_income
    "income_household_under_5",
    "income_household_5_to_10",
    "income_household_10_to_15", 
    "income_household_15_to_20",
    "income_household_20_to_25",
    "income_household_25_to_35",
    "income_household_35_to_50",
    "income_household_50_to_75",
    "income_household_75_to_100",
    "income_household_100_to_150",
    "income_household_150_over", 
    "income_individual_median",
    "metastatic_diagnosis_period"
]

socioeconomic_factors = [
    "poverty",
    "rent_burden",
    "metastatic_diagnosis_period"
]

education = [
    "education_less_highschool",
    "education_highschool",
    "education_some_college",
    "education_bachelors",
    "education_graduate",
    "education_stem_degree",
    "metastatic_diagnosis_period"
]

employment = [
    "self_employed",
    "farmer",
    "metastatic_diagnosis_period"
]

other = [
    "disabled",
    "limited_english",
    "commute_time",
    "health_uninsured",
    "veteran",
    "metastatic_diagnosis_period"
]

# Time-based averages (needs further handling based on your approach)
time_based_averages = [
    'Average of Jan-13', 'Average of Feb-13', 'Average of Mar-13', 'Average of Apr-13', 'Average of May-13', 'Average of Jun-13', 'Average of Jul-13', 'Average of Aug-13', 'Average of Sep-13', 'Average of Oct-13', 'Average of Nov-13', 'Average of Dec-13', 'Average of Jan-14', 'Average of Feb-14', 'Average of Mar-14', 'Average of Apr-14', 'Average of May-14', 'Average of Jun-14', 'Average of Jul-14', 'Average of Aug-14', 'Average of Sep-14', 'Average of Oct-14', 'Average of Nov-14', 'Average of Dec-14', 'Average of Jan-15', 'Average of Feb-15', 'Average of Mar-15', 'Average of Apr-15', 'Average of May-15', 'Average of Jun-15', 'Average of Jul-15', 'Average of Aug-15', 'Average of Sep-15', 'Average of Oct-15', 'Average of Nov-15', 'Average of Dec-15', 'Average of Jan-16', 'Average of Feb-16', 'Average of Mar-16', 'Average of Apr-16', 'Average of May-16', 'Average of Jun-16', 'Average of Jul-16', 'Average of Aug-16', 'Average of Sep-16', 'Average of Oct-16', 'Average of Nov-16', 'Average of Dec-16', 'Average of Jan-17', 'Average of Feb-17', 'Average of Mar-17', 'Average of Apr-17', 'Average of May-17', 'Average of Jun-17', 'Average of Jul-17', 'Average of Aug-17', 'Average of Sep-17', 'Average of Oct-17', 'Average of Nov-17', 'Average of Dec-17', 'Average of Jan-18', 'Average of Feb-18', 'Average of Mar-18', 'Average of Apr-18', 'Average of May-18', 'Average of Jun-18', 'Average of Jul-18', 'Average of Aug-18', 'Average of Sep-18', 'Average of Oct-18', 'Average of Nov-18', 'Average of Dec-18',
    'metastatic_diagnosis_period'
]  # List to store features based on your chosen approach

#### Patient Demographics

In [ ]:
# Calculate correlation coefficients
correlation_matrix = df_train[patient_demographics].corr()

# Create a heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')  # Adjust 'cmap' for color scheme
plt.title('Correlation Matrix (Features vs. Target)')
plt.show()


####  Demographics by Zip Code

In [ ]:
# Calculate correlation coefficients
correlation_matrix = df_train[demographics_by_zip_code].corr()

# Create a heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')  # Adjust 'cmap' for color scheme
plt.title('Correlation Matrix (Features vs. Target)')
plt.show()

#### race_and_ethnicity 

In [ ]:
# Calculate correlation coefficients
correlation_matrix = df_train[race_and_ethnicity].corr()

# Create a heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')  # Adjust 'cmap' for color scheme
plt.title('Correlation Matrix (Features vs. Target)')
plt.show()

#### Age group correlation 

In [ ]:
# Calculate correlation coefficients
correlation_matrix = df_train[age_groups].corr()

# Create a heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')  # Adjust 'cmap' for color scheme
plt.title('Correlation Matrix (Features vs. Target)')
plt.show()

####   marital_status

In [ ]:
# Calculate correlation coefficients
correlation_matrix = df_train[marital_status].corr()

# Create a heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')  # Adjust 'cmap' for color scheme
plt.title('Correlation Matrix (Features vs. Target)')
plt.show()


### Income Correlation

In [ ]:
# Calculate correlation coefficients
correlation_matrix = df_train[income].corr()

# Create a heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')  # Adjust 'cmap' for color scheme
plt.title('Correlation Matrix (Features vs. Target)')
plt.show()

#### Socio-economic factors

In [ ]:
# Calculate correlation coefficients
correlation_matrix = df_train[socioeconomic_factors].corr()

# Create a heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')  # Adjust 'cmap' for color scheme
plt.title('Correlation Matrix (Features vs. Target)')
plt.show()


#### Education

In [ ]:
# Calculate correlation coefficients
correlation_matrix = df_train[education].corr()

# Create a heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')  # Adjust 'cmap' for color scheme
plt.title('Correlation Matrix (Features vs. Target)')
plt.show()

#### Employment

In [ ]:
# Calculate correlation coefficients
correlation_matrix = df_train[employment].corr()

# Create a heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')  # Adjust 'cmap' for color scheme
plt.title('Correlation Matrix (Features vs. Target)')
plt.show()


#### Other

In [ ]:
# Calculate correlation coefficients
correlation_matrix = df_train[other].corr()

# Create a heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')  # Adjust 'cmap' for color scheme
plt.title('Correlation Matrix (Features vs. Target)')
plt.show()

### Correlation between each features and the target using .corr() with spearman and pearson methods

In [ ]:
df_train.corr(method='spearman')[target_col].abs().sort_values()  # Spearman correlation for non-normal data

In [ ]:
df_train.corr()[target_col].abs().sort_values() # Pearson is the default

### Feature engineering

Creating new features in data science is an essential process that involves generating synthetic attributes from existing data. This process aims to extract additional information from the existing dataset, which can help improve the performance of machine learning models. By creating new features, data scientists can capture complex relationships and patterns that may be overlooked with only the original features.

# Regression modeling

**Regression Modelling in Python with Scikit-learn**

In this section, we will explore regression modelling using Python and the popular machine learning library, Scikit-learn. Regression is a supervised learning task that aims to model the relationship between input variables (features) and a continuous output variable (target).

Scikit-learn provides a wide range of regression algorithms, including linear regression, random forests, catboost, and XGBoost. We will cover the basics of each algorithm and demonstrate how to implement them using Scikit-learn.

**Linear Regression**

Linear regression is a simple yet powerful regression algorithm that models the relationship between features and the target as a linear function. We will discuss the assumptions of linear regression, how to train a model, and how to evaluate its performance using the root mean squared error (RMSE).

**Random Forest**

Random forests are an ensemble learning method that combines multiple decision trees to improve prediction accuracy and reduce overfitting. Each tree in the forest is trained on a random subset of the data, and the final prediction is made by averaging the predictions of all the trees. Random forests are particularly useful for handling complex relationships and interactions between features, and they are also robust to outliers and noise in the data. We will demonstrate how to use random forests to model complex relationships and how to tune hyperparameters such as the number of trees and the maximum depth of the trees.

**CatBoost**

CatBoost is a gradient boosting algorithm that is known for its speed and accuracy. It uses a novel algorithm called "ordinal encoding" to handle categorical variables, which allows it to handle large numbers of categories efficiently. However, CatBoost can also be used for regression tasks that do not involve categorical data. We will demonstrate how to use CatBoost to model complex relationships and how to tune hyperparameters such as the learning rate and the number of iterations.

**XGBoost**

XGBoost is a gradient boosting algorithm that is known for its speed and accuracy in regression tasks. It uses a novel algorithm called "tree boosting" to combine multiple decision trees, which allows it to handle complex relationships and interactions between features. We will demonstrate how to use XGBoost to model complex relationships and how to tune hyperparameters such as the learning rate, the number of trees, and the maximum depth of the trees.

**Model Selection and Hyperparameter Tuning**

In addition to the regression algorithms themselves, Scikit-learn provides tools for model selection and hyperparameter tuning. We will discuss the importance of model selection and hyperparameter tuning, and demonstrate how to use Scikit-learn's GridSearchCV and RandomizedSearchCV classes to find the best set of hyperparameters for a given regression algorithm.


In [ ]:
#pip install catboost

In [ ]:
#!pip install xgboost

In [ ]:
#pip install shap

In [ ]:
# Import libraries

import catboost as cb
import numpy as np
from numpy import absolute
import pandas as pd
import seaborn as sns
#import shap
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style="whitegrid")
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from sklearn.inspection import permutation_importance
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [ ]:
!pwd

## Separate training and test data from df_train

In [ ]:
# Create X and y dataframes

target_column = 'metastatic_diagnosis_period'

X = df_train.drop(target_column, axis=1)
y = df_train[target_column]

X.shape

In [ ]:
y.shape

## Create train/test splits

**train_test_split** is a common method for splitting data into training and testing sets. However, it does not handle class imbalance well and may result in uneven class distributions in the training and testing sets.

**StratifiedShuffleSplit** is a cross-validation method that can be used when the target variable has a significant class imbalance. It randomly divides the data into training and testing sets, ensuring that the proportions of classes in both sets remain the same as those in the original data. This is useful when dealing with data where one class represents a minority or outlier.

Below we will demonstrate how to use each approach. **You will only use one or the other, not both.**

Let's try using train_test_split first, then come back and try stratified_train_test_split

### train_test_split

Use this or StratifiedShuffleSplit, not both

In [ ]:
# Create train/test splits using train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=5)

X_train.shape

### StratifiedShuffleSplit

Use this or train_test_split, not both

In [ ]:
def split_X_y(df, column_name, row_idx=None):
    columns = df.columns.values.tolist()
    columns.remove(column_name)
    if row_idx is None:
        return df[columns], df[column_name]
    else:
        return df[columns].iloc[row_idx], df[column_name].iloc[row_idx]

In [ ]:
def stratified_train_test_split(df):
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=1)
    train_idx, test_idx = next(sss.split(*split_X_y(df, target_column)))
    X_train, y_train = split_X_y(df, target_column, train_idx)
    X_test, y_test = split_X_y(df, target_column, test_idx)

    return X_train, y_train, X_test, y_test

In [ ]:
X_train, y_train, X_test, y_test = stratified_train_test_split(df_train)

## Function to predict on df_test data and create csv for Kaggle submission

In [ ]:
def predict_test_data(model, df_test):
    y_pred = model.predict(df_test)
    df_test['metastatic_diagnosis_period'] = y_pred.tolist()
    answers = df_test['metastatic_diagnosis_period']
    answers.to_csv("output/answers.csv")
    return

## Linear Regression

#### - Linear regression: Guessing a continuous value using a straight line based on features. Easy to understand!
#### - Strengths: Easy to interpret, works fast, good starting point for many problems.
#### - Weaknesses: Only works for straight line relationships, dislikes outliers.
#### - Used for: Predicting stuff in finance, science, and machine learning! (e.g. house prices)

In [ ]:
# Create the linear regression model
model = LinearRegression()

In [ ]:
model.fit(X_train,y_train)

In [ ]:
# Evaluate the model using RMSE
rmse = np.sqrt(np.mean((y_test - model.predict(X_test)) ** 2))
print("RMSE:", rmse)

In [ ]:
# Plot actual vs. predicted values
plt.figure(figsize=(8, 6))
plt.scatter(y_test, model.predict(X_test), label="Predictions", color="blue")
plt.plot(y_test, y_test, label="Actual", color="red")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
title = "Actual vs. Predicted Values (RMSE: {:.2f})".format(rmse)
plt.title(title)
plt.legend()
plt.grid(True)
plt.show()

print("RMSE:", rmse)  # Print RMSE for reference

### Predict on test data and generate csv for Kaggle submission

In [ ]:
#predict_test_data(model, df_test)

## Random Forest

#### - Ensemble method: Random Forest combines multiple decision trees, reducing variance and improving overall accuracy.
#### - Randomization: During training, each tree considers a random subset of features at each split point, preventing overfitting.
#### - Out-of-bag error: Random Forest provides an estimate of prediction error for unseen data using out-of-bag samples (data points not used to train a specific tree).
#### - Feature importance: Random Forest calculates the importance of each feature based on how much it contributes to the splits in the trees, aiding in feature selection.

In [ ]:
model = RandomForestRegressor(n_estimators=500, max_features = 'sqrt', max_depth = 8, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('RMSE: ', rmse)

In [ ]:
# Plot actual vs. predicted values
plt.figure(figsize=(8, 6))
plt.scatter(y_test, model.predict(X_test), label="Predictions", color="blue")
plt.plot(y_test, y_test, label="Actual", color="red")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
title = "Actual vs. Predicted Values (RMSE: {:.2f})".format(rmse)
plt.title(title)
plt.legend()
plt.grid(True)
# plt.show()
plt.savefig("./RandomForest_actual_vs_predicted1.png")


print("RMSE:", rmse)  # Print RMSE for reference

In [ ]:
param_grid = {
   'n_estimators': [100, 200, 500],
   'max_features': ['auto', 'sqrt', 'log2'],
   'max_depth' : [4,5,6,7,8]
}
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)
grid_search.best_params_ 

In [ ]:
model = RandomForestRegressor(n_estimators=500, max_features = 'sqrt', max_depth = 8, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
print('RMSE: ', rmse)

### Predict on test data and generate csv for Kaggle submission

In [ ]:
# predict_test_data(model, df_test)

## XGBoost

XGBoost is a gradient boosting algorithm that is known for its speed and accuracy in regression tasks.

#### - Gradient Boosting: XGBoost follows a gradient boosting approach, where each tree learns from the errors of the previous one, leading to a more optimized model.
#### - Regularization: It incorporates L1 and L2 regularization to penalize model complexity and prevent overfitting.
#### - Parallelization: XGBoost is optimized for parallel and distributed computing, making it efficient for large datasets.
#### - Early Stopping: XGBoost can implement early stopping, which halts training when the validation score doesn't improve for a certain number of iterations, preventing overfitting

In [ ]:
# Define the model
model = XGBRegressor()

In [ ]:
# Define the model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

In [ ]:
# Evaluate the model using cross validation
scores = cross_val_score(model, X_test, y_test, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1)

In [ ]:
print("Cross-validation Mean Squared Error: ", scores.mean())

In [ ]:
# Force scores to be positive
scores = absolute(scores.mean())

In [ ]:
model.fit(X_train, y_train)

In [ ]:
rmse = np.sqrt(scores)

print('Mean RMSE: %.3f' % (rmse.mean()))

In [ ]:
# Plot actual vs. predicted values
plt.figure(figsize=(8, 6))
plt.scatter(y_test, model.predict(X_test), label="Predictions", color="blue")
plt.plot(y_test, y_test, label="Actual", color="red")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
title = "Actual vs. Predicted Values (RMSE: {:.2f})".format(rmse)
plt.title(title)
plt.legend()
plt.grid(True)
# plt.show()
plt.savefig("./RandomForest_actual_vs_predicted1.png")


print("RMSE:", rmse)  # Print RMSE for reference

### Predict on test data and generate csv for Kaggle submission

In [ ]:
#predict_test_data(model, df_test)

## CatBoost

CatBoost is a gradient boosting algorithm that is known for its speed and accuracy.

#### - Ordered CatBoosting: CatBoost handles categorical features by treating them as ordered (if applicable) or using a sophisticated approach for unordered categories.
#### - Leave-One-Out Ranking: CatBoost employs a leave-one-out ranking objective during training, focusing on improving the relative order of predictions for similar data points.
#### - Symmetric Trees: CatBoost utilizes symmetric trees, which can be split from either left or right, potentially leading to more efficient training.
#### - Feature Importance Scores: Similar to Random Forest, CatBoost provides feature importance scores to understand which features contribute most to the model's predictions.

In [ ]:
'''
Here's an overview of what the code does:

cb is a shorthand for the CatBoost module.
Pool is a class in CatBoost that represents a dataset. It takes two arguments: the feature matrix (X_train and X_test) and the target vector 
(y_train and y_test). The two Pool objects are created using the feature matrices and target vectors. These objects will be used to train 
and evaluate the model.

So, in summary, the code creates two Pool objects, which represent the training and testing datasets, respectively.
'''

train_dataset = cb.Pool(X_train, y_train) 
test_dataset = cb.Pool(X_test, y_test)

In [ ]:
# Create the CatBoost regression model using RMSE as the loss function

model = cb.CatBoostRegressor(loss_function='RMSE')

In [ ]:
'''
This Python code snippet demonstrates how to use the GridSearchCV class from the scikit-learn library to perform hyperparameter tuning 
for a regression model. The GridSearchCV class takes a grid of hyperparameters and trains the model with different combinations of 
these hyperparameters.

In this example, the grid contains four hyperparameters: iterations, learning_rate, depth, and l2_leaf_reg. The iterations hyperparameter 
controls the number of iterations the model will run for, the learning_rate hyperparameter controls the learning rate of the model, the 
depth hyperparameter controls the maximum depth of the trees in the model, and the l2_leaf_reg hyperparameter controls the regularization 
strength of the model.

The GridSearchCV class then trains the model with different combinations of these hyperparameters and evaluates the performance of the 
model using the mean squared error metric. The best combination of hyperparameters is then selected based on the performance of the model.
'''

grid = {'iterations': [100, 150, 200],
        'learning_rate': [0.03, 0.1],
        'depth': [2, 4, 6, 8],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]}

model.grid_search(grid, train_dataset, verbose=1)

# Compare GridSearchCV results to CatBoost grid_search
#grid_search = GridSearchCV(model, grid, cv=5, scoring='neg_mean_squared_error')
#grid_search.fit(X_train, y_train)

Now that we have the best model from the grid search, we will apply it to our test dataset.

The RMSE metric measures the average squared difference between the predicted values and the actual values. A lower RMSE indicates a better fit.

The R-squared metric measures the proportion of the variance in the target variable that is explained by the model. An R-squared value of 1 indicates that the model explains all of the variance in the target variable, while a value of 0 indicates that the model does not explain any of the variance.


In [ ]:
'''
The predict method is used to make predictions on the test dataset, and the results are stored in the pred variable. It is important 
to note that the predict method should only be used on a test dataset that has not been used for training the model. Using the predict 
method on a training dataset can lead to overfitting and poor generalizability of the model.

The metrics RMSE and R2 are then calculated.
'''

pred = model.predict(X_test)
rmse = (np.sqrt(mean_squared_error(y_test, pred)))
r2 = r2_score(y_test, pred)

print('Testing performance')
print('RMSE: {:.2f}'.format(rmse))
print('R2: {:.2f}'.format(r2))

### Predict on the real test data and create the csv for Kaggle submission

In [ ]:
#predict_test_data(model, df_test)

### Submit results to Kaggle!

### CatBoost feature importance

In [ ]:
feature_names = X.columns
print(feature_names)

In [ ]:
sorted_feature_importance = model.feature_importances_.argsort()
plt.barh(feature_names[sorted_feature_importance], 
        model.feature_importances_[sorted_feature_importance], 
        color='turquoise')
plt.xlabel("CatBoost Feature Importance")